In [1]:
from iotbx.data_manager import DataManager
from iotbx.map_model_manager import map_model_manager as MapModelManager
from libtbx.test_utils import approx_equal

In [2]:
from phenix.programs.ligands_cc import LigandsCC, LigandSelection

# FileIO

In [10]:
model_file = "../data/6qi8_4552/6qi8_4552.cif"
map_file = "../data/6qi8_4552/6qi8_4552.map"
assert(os.path.exists(model_file))
assert(os.path.exists(map_file))
dm = DataManager()
dm.process_model_file(model_file)
dm.process_real_map_file(map_file)
model = dm.get_model()

# this is necessary for cryo-em models without crystal symmetry
if model.crystal_symmetry() is None:
    from cctbx.maptbx.box import shift_and_box_model
    model = shift_and_box_model(model)

mmm = MapModelManager(map_manager=dm.get_real_map(), model=dm.get_model())


# Ligand processing

In [22]:
# Instantiate the class
ligands_cc = LigandsCC(mmm, resolution=3.5) 

# The validation function calculates whole map/model CCmask
ligands_cc.validate()
print(ligands_cc.ccmask_full_model)

0.0229708559444


In [23]:
# extract ligands and calculate ligand ccmask
ligands_cc.process_ligands()
print(ligands_cc.ccmask_ligands)

[-0.080566400250404, -0.05039665095274361, 0.02568153796222316, -0.058698045705798986, 0.10242327846306232, -0.07061912170822986]


In [40]:
# optionally manipulate ligands
ligand_models = [mmmlig.model() for mmmlig in ligands_cc.ligand_map_model_managers]
ligand0 = ligand_models[0]
for rg in ligand0.get_hierarchy().residue_groups():
    print(rg.unique_resnames()[0])

ADP


## Write output

In [24]:
# write ouput
output_dir = "../data/output"

for mmmlig in ligands_cc.ligand_map_model_managers:
    ligand_name = LigandsCC.ligand_name(mmmlig.model())
    ligand_model_file = os.path.join(output_dir,ligand_name)
    ligand_map_file = os.path.join(output_dir,ligand_name+".map")
    
    dm.write_model_file(mmmlig.model(),
                          filename=ligand_model_file,
                          extension="cif",
                          overwrite=True)
    
    dm.write_real_map_file(mmmlig.map_manager(),
                          filename=ligand_map_file,
                          overwrite=True)